# Window sampling
This notebook is responsible for the synchronisation of the physiological and video dataframes created in the previous notebooks. It is also responsible for getting the data samples from these synchronised dataframes, by running a smaller sliding window over these DataFrames and then computing various features and target variables on the selected windows. In the first few cells, some functions are described which implement these different tasks, like getting the different start and end points of the windows and also computing the targets and features. The last two cells are responsible for the actual sampling of all the different DataFrames.
#### Requirements
If one wants to run this notebook make sure you have created the physiological and video DataFrames, which can be done by running the previous notebooks `1-ak-physio-dataframe` and `2-ak-video-dataframe`. If executed properly, these notebooks should have created two DataFrames, one for physiological signals and one for video signals, for each participant. These DataFrames should be stored in the `data\interim` directory, in either the `video` or `physiological` subdirectory.

Besides the DataFrames, one also needs to have the scripts that contain the functions responsible for the target and feature computation, `targetComputation.py` and `features.py`. These should be in the same directory as the notebooks.

Finally, to run the function in from the `targetComputation.py` script one also needs to have run the `3-ak-target-variable` notebook, which is responsible for the csv files that contain information about physiological signals in the baseline component. These should be stored in the `data/information` directory.

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from targetComputation import compute_EDA_Targets, compute_ECG_Targets
import features as ft
from CONSTANTS import *

After loading in the required modules, we store the working directory in a variable called `project_dir`. We then store the folder where the data files are located in a variable called `data_dir`. We also store the location of the subdirectories, which store the video and physiological dataframes in, respectively, `video_dir` and `physio_dir`. We store the video and physiological filenames in different variables. We also create a variable `pps`, which stores the pp ids for the pp for which both the video and physiological dataframes are available.

In [4]:
project_dir = os.getcwd().split('\\')[:-1]
project_dir = '\\'.join(project_dir) # Get the project dir
# Get the data dir
if Mitchel:
    data_dir = 'C:\\Users\\mitch\\OneDrive - UGent\\UGent\\Projects\\7. tDCS_Stress_WM_deSmet\\data'
    data_dir = 'Z:\\ghep_lab\\2020_DeSmetKappen_tDCS_Stress_WM_VIDEO\\Data'
else: 
    data_dir = project_dir + '\\data'

video_dir = data_dir+'\\interim\\video' # Get the video dir
physio_dir = data_dir+'\\interim\\physiological' # Get the physio dir
video_files = [file for file in os.listdir(video_dir)] # Get the video files
physio_files = [file for file in os.listdir(physio_dir)] # Get the physio files

pps = list(set([file[:-8] for file in video_files]).intersection(set([file[:-8] for file in physio_files]))) # Get the intersection of the pp ids in the video files and the pp ids from the physio files
pps = sorted([int(pp) for pp in pps]) # Set all the pp ids to int type and sort this list

## Functions for sampling
After loading the required modules and storing the different location of the data files in variables, we implemented some functions that will perform the sampling of all the DataFrames further below.
### Getting the start and end points of a single DataFrame
First off the `get_start_end` function. This function finds all the start and end points of a sample in a given video DataFrame, given a window and step size. This function works with any desired window and step size, as longs as they are integers. The function returns a list of tuples, with each tuple reflecting the start and end point of a window.

In [5]:
def get_start_end(data: pd.DataFrame, window_size: int, step_size: int) -> list:
    """Get start & end points of a video signal DataFrame on a given window size and step size.
    Returns a list of tuples (startpoint, endpoint)
    """
    points = [] # Create the list to store the points in
    start = data.t_from_start.values[0] # Get the first frame, which acts as the starting point of the first frame
    end = start + window_size # Compute the first end point, by adding the desired window size to the starting point
    points.append((start, end)) # Add the start and end point to the list of points

    while end+step_size < data.t_from_start.values[-1]: # While the end point + the step size does not reach beyond the last frame
        start += step_size # Update the new starting point, by adding the step size to the last starting point 
        end += step_size # Update the new ending point, by adding the step size to the last ending point 
        points.append((start, end)) # Add this new start and end point to the list of points
    
    return points # Return the list of points

### Getting one window
The function `get_window` subsets one window/sample from a dataframe, based on a given start and end point. Combined with the `get_start_end` function, this functions helps us to gather all the possible windows/samples from a dataframe.

In [6]:
def get_window(data: pd.DataFrame, start: int, end: int) -> pd.DataFrame:
    """Return subset of a DataFrame (window) between a certain start and endpoint."""
    return data.loc[(data.t_from_start >= start) & (data.t_from_start <= end), :] # Get all the rows after (or equal to) the starting point AND (or equal to) the ending point.

### Processing the windows
Below we have implemented two functions for the processing of respectively a video window and a physiological window. The functions `process_physio_window` and `process_video_window` have as input a window/sample (a subset of DataFrame obtained through the function `get_window`) and compute various features or targets, which are returned in a dict. When we want to compute extra or other features/targets, we can just add the computation in these functions.

In [7]:
def process_physio_window(window: pd.DataFrame) -> dict:
    """Processes the physiological window. Returns a dict containing the computed target variables and the first and last point of the window."""
    processed_window = {} # Create the dict to store the target variables and other information in
    
    processed_window['t_start_physio'] = window.t_from_start.values[0] # Get the starting timestamp of this window and add this to the dict
    processed_window['t_end_physio'] = window.t_from_start.values[-1] # Get the ending timestamp of this window and add this to the dict
    
    processed_window = {**processed_window, **compute_EDA_Targets(window)} # Compute the EDA targets by using the function from targetComputation.py and add this to dict
    processed_window = {**processed_window, **compute_ECG_Targets(window)} # Compute the ECG targets by using the function from targetComputation.py and add this to dict
    
    return processed_window # Return the processed physio window

In [8]:
def process_video_window(window: pd.DataFrame) -> dict:
    """Processes the video window. Returns a dict containing the computed features and the first and last frame of the window."""
    processed_window = {} # Create the dict to store the target variables and other information in
    
    processed_window['t_start_video'] = window.t_from_start.values[0] # Get the starting frame of this window and add this to the dict
    processed_window['t_end_video'] = window.t_from_start.values[-1] # Get the ending frame of this window and add this to the dict
    
    ## Below we calculate various features using the functions from features.py, and adding these to the dict
    processed_window = {**processed_window, **ft.compute_mean_AUs(window)} # Computation of mean FAU and mean change FAU
    processed_window = {**processed_window, **ft.compute_std_AUs(window)} # Computation of std FAU
    processed_window = {**processed_window, **ft.compute_arousal(window)} # Computation of arousal
    processed_window = {**processed_window, **ft.compute_emotions(window)} # Computation of various emotions
    processed_window = {**processed_window, **ft.compute_head_motion(window)} # Computation of std of head motion in different directions
    processed_window = {**processed_window, **ft.compute_PD_features(window)} # Computation of various Pupil Diameter (PD) features
    processed_window['blink_rate'] = ft.compute_blink_rate(window) # Computation of the blink rate
    
    return processed_window # Return the processed physio window

### Checking the quality of the video window
Finally we have created a function that assess the quality of a given video window (subset of video DataFrame). Based on defined rules, it either returns false if the rules are broken, thus the video window is of too low quality to be used as a data sample, or it returns true if none of the rules are broken. If one wants to change the rules on which the decision is made to exclude video data samples, one can do so by changing this function.

In [9]:
def check_video_window(window: pd.DataFrame) -> bool:
    """Checks the video window, based on the defined rules. If the rule is broken we return false and do not use the entire window. Else we return true"""
    global removed # Get access to the variable removed, defined later on, which allows tracking of the amount of removed/declined windows
    if (window.confidence >= 0.8).sum()/len(window.confidence) < 0.95: # If less than 95% of the frames in the video window got confidence rating below 80% 
        removed += 1 # Remove this window, thus adding 1 to the removed variable
        return False # And returning False
    else: # If no rules were broken
        return True # Return True

#### Sampling and processing one participant
Below we have implemented the `sample_pp` function, which samples and processes the data of one participant. After reading in the physio and video DataFrame of the participant, it implements the four functions implemented in the previous cells, to sample and process the video and physiological DataFrames. It stores the processed windows in a list of dicts, and returns this at the end of the function.

In [10]:
def sample_pp(physio_data: pd.DataFrame, video_data: pd.DataFrame, window_size: int, step_size: int) -> list:
    """Samples the data of one specific participant. Returns a list of dicts, where each dict represents a processed window"""
    points = get_start_end(video_data, window_size, step_size) # Get the starting and end points based on the video DataFrame
    processed_windows = [] # Create an empty list in which to store the dicts
    i = 1 # i represents the ith window of the participant
    for point in points: # for each start end point tuple
        start, end = point # Get the start and end point
        video_window = get_window(video_data, start, end) # Get the video window
        if check_video_window(video_window): # If this video window passes the quality checks
            physio_window = get_window(physio_data, start, end) # Also get the physio window
            
            processed_physio_window = process_physio_window(physio_window) # Process the physio window
            processed_video_window = process_video_window(video_window) # Process the video window

            processed_window = {**processed_video_window, **processed_physio_window} # Add these processed windows together
            processed_window['start'] = start # Add the starting point to the processed window dict
            processed_window['end'] = end # Add the ending point to the processed window dict
            processed_window['pp'] = pp # Add the pp id to the processed window dict
            processed_window['pp_window'] = i # Add the window id of this specific pp to the processed window dict 
            
            processed_windows.append(processed_window) # Add the processed window to the list of processed windows
        i += 1 # Update the window id
    return processed_windows # Return the list of processed windows

## Sampling the data 
The following two cells process the data from all the participants, using the above implemented functions.
### Loading in the participants
In order to speed up the code, we first load in all the DataFrames, and storing these dataframes in dicts, respectively `pp_video` and `pp_physio`. These dicts can be accessed using the participants id. Doing this, takes up a lot of memory, however speeds up the sampling process. If we were to have more than 100 participants, or larger DataFrames, we would however run into memory issues.

There are also 2 lines of codes that set a subset of the video dataframe to nan, if those rows have a confidence rating lower then 80%. This is necessary for the function `check_video_window` to work.

In [15]:
pp_physio = {} # Create a dict to store the physio DataFrames in
pp_video = {} # Create a dict to store the video DataFrames in

for pp in tqdm(pps, desc='pp', leave=False): # For each of the pp ids
    pp_physio[pp] = pd.read_feather(f'{physio_dir}\\{pp}.feather') # Read in physio DataFrame and store this in dict
    df = pd.read_feather(f'{video_dir}\\{pp}.feather') # Read in video DataFrame
    cols = [col for col in df.columns if col not in ['frame', 'face_id', 'timestamp', 'confidence', 'success', 'started', 'pp', 't_from_start', 'frames_away_start ']] # Get all the cols that need to be set to nan if < 80%
    df.loc[df.confidence<0.8, cols] = np.nan # Set desired cols to nan if confidence < 80%
    pp_video[pp] = df # Store DataFrame to dict

pp:   0%|          | 0/67 [00:00<?, ?it/s]

### Processing all the participants
Below we sample all the participants and stored the processed windows in a dataframe, and saving it to the `data\processed` directory. These DataFrames will be used for the modelling steps in the next notebooks. The variables `window_sizes` and `step_sizes` contain the step and window sizes to sammple DataFrames with. `window_sizes` is a list of ints, where each element represent a window size in seconds. `step_sizes` is a list of floats, where each float is used as a proportion of the current window size to compute the step size: `step_size` = `window_size` * `float`.

In [16]:
pd.set_option('mode.chained_assignment',None) # Set this options to avoid annoying warnings (Not necessary)
window_sizes = [60*3, 60*5] # List of window sizes in seconds
step_sizes = [1] # List of step sizes proportionally to seconds

for window_size in tqdm(window_sizes, desc='windows'): # For each window size
    for step_size in tqdm(step_sizes, desc='steps', leave=False): # For each step size
        df = [] # Create a list in which to store the processed windows
        step_size *= window_size # Compute the step size
        step_size = int(step_size) # Set it to an integer
        removed = 0 # Create this to keep track of how many windows are removed
        for pp in tqdm(pps, desc='pp', leave=False): # For each participant
            df += sample_pp(pp_physio[pp], pp_video[pp], window_size, step_size) # Sample and process all the windows and add this to list of processed windows
        df = pd.DataFrame(df) # Create a DataFrame from this list of dicts
        df.pp = df['pp'].astype('str') # Set pp to type string
        df['window'] = df.index # Set the index as column `window` to have unique window ids for all the windows
        print(f'Finished window size: {window_size} step size: {step_size}. Sampled a total of {len(df.pp)} windows. Removed total of {removed} windows, ~{int(((removed)/(removed+len(df.pp)))*100)} percent of all possible windows.') # Print the result of this window and step size
        df.to_feather(f"{data_dir}\\processed\\window_{window_size}_step_{step_size}.feather") # Save the DataFrame to desired folder

windows:   0%|          | 0/2 [00:00<?, ?it/s]

steps:   0%|          | 0/1 [00:00<?, ?it/s]

pp:   0%|          | 0/67 [00:00<?, ?it/s]

Finished window size: 180 step size: 180. Sampled a total of 178 windows. Removed total of 158 windows, ~47 percent of all possible windows.


steps:   0%|          | 0/1 [00:00<?, ?it/s]

pp:   0%|          | 0/67 [00:00<?, ?it/s]

Finished window size: 300 step size: 300. Sampled a total of 98 windows. Removed total of 100 windows, ~50 percent of all possible windows.
